<a href="https://colab.research.google.com/github/DjonatanS/Algoritmo-DataStructure-Python/blob/main/Postgres_BITRIXapi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from time import sleep
from urllib.parse import urlparse
import pandas as pd
import psycopg2
import numpy as np
import xlrd
import time
import datetime
from sqlalchemy import create_engine
import urllib.parse
import datetime


/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
#variables
minutos = 60
hora = minutos * 60
horario = datetime.datetime.now().strftime("%H:%M:%S")
horaInt = int(horario.split(":")[0])

API BITRIX

In [3]:
class Bitrix24(object):
    """A user-created :class:`Bitrix24 <Bitrix24>` object.
    Used to sent to the server.

    :param domain: REST call domain, including account name, user ID and secret code.
    :param timeout: (Optional) waiting for a response after a given number of seconds.
    Usage::
      >>> from bitrix24 import Bitrix24
      >>> bx24 = Bitrix24('https://example.bitrix24.com/rest/1/33olqeits4avuyqu')
      >>> bx24.callMethod('crm.product.list')
    """

    def __init__(self, domain, timeout=60):
        """Create Bitrix24 API object
        :param domain: str Bitrix24 webhook domain
        :param timeout: int Timeout for API request in seconds
        """
        self.domain = self._prepare_domain(domain)
        self.timeout = timeout

    def _prepare_domain(self, domain):
        """Normalize user passed domain to a valid one."""
        if domain == '' or not isinstance(domain, str):
            raise Exception('Empty domain')

        o = urlparse(domain)
        user_id, code = o.path.split('/')[2:4]
        return "{0}://{1}/rest/{2}/{3}".format(o.scheme, o.netloc, user_id, code)

    def _prepare_params(self, params, prev=''):
        """Transforms list of params to a valid bitrix array."""
        ret = ''
        if isinstance(params, dict):
            for key, value in params.items():
                if isinstance(value, dict):
                    if prev:
                        key = "{0}[{1}]".format(prev, key)
                    ret += self._prepare_params(value, key)
                elif (isinstance(value, list) or isinstance(value, tuple)) and len(value) > 0:
                    for offset, val in enumerate(value):
                        if isinstance(val, dict):
                            ret += self._prepare_params(
                                val, "{0}[{1}][{2}]".format(prev, key, offset))
                        else:
                            if prev:
                                ret += "{0}[{1}][{2}]={3}&".format(
                                    prev, key, offset, val)
                            else:
                                ret += "{0}[{1}]={2}&".format(key, offset, val)
                else:
                    if prev:
                        ret += "{0}[{1}]={2}&".format(prev, key, value)
                    else:
                        ret += "{0}={1}&".format(key, value)
        return ret

    def callMethod(self, method, **params):
        """Calls a REST method with specified parameters.

       :param url: REST method name.
       :param \*\*params: Optional arguments which will be converted to a POST request string.
       :return: Returning the REST method response as an array, an object or a scalar
       """

        try:
            url = '{0}/{1}.json'.format(self.domain, method)

            p = self._prepare_params(params)

            if method.rsplit('.', 1)[0] in ['add', 'update', 'delete', 'set']:
                r = requests.post(url, data=p, timeout=self.timeout).json()
            else:
                r = requests.get(url, params=p, timeout=self.timeout).json()
        except ValueError:
            if r['error'] not in 'QUERY_LIMIT_EXCEEDED':
                raise BitrixError(r)
            # Looks like we need to wait until expires limitation time by Bitrix24 API
            sleep(2)
            return self.callMethod(method, **params)

        if 'error' in r:
            raise BitrixError(r)
        if 'start' not in params:
            params['start'] = 0
        if 'next' in r and r['total'] > params['start']:
            params['start'] += 50
            data = self.callMethod(method, **params)
            if isinstance(r['result'], dict):
                result = r['result'].copy()
                result.update(data)
            else:
                result = r['result'] + data
            return result
        return r['result']

CONEXÕES

In [23]:
#BITRIX 
apiFaz = Bitrix24('https://fazcapital.bitrix24.com.br/rest/1471/rn9tlazdexortpli/', 60)

#AMAZON
aws = psycopg2.connect(
    host="teste.cpdbmdlpalvb.us-east-1.rds.amazonaws.com",
    database="etl_fazcapital",
    user="fazcapital",
    password="A1b2c3d4!#$%&*",
    port="5432")


In [26]:
    
while horaInt >= 10 and horaInt <= 18:  
        aws = psycopg2.connect(
        host="teste.cpdbmdlpalvb.us-east-1.rds.amazonaws.com",
        database="etl_fazcapital",
        user="fazcapital",
        password="A1b2c3d4!#$%&*",
        port="5432")
        
        
                
        telefone = pd.read_sql("SELECT id FROM telefone", aws)
        telefone = telefone['id'].astype(int)
        idMaximoTelefone =  int(telefone.max())
        

        


    
        bitrixTelephony = apiFaz.callMethod('voximplant.statistic.get', next = True,
                                    filter = {'>ID': idMaximoTelefone})
            

        DataBitrix = pd.DataFrame(bitrixTelephony)
        DataBitrix.columns = DataBitrix.columns.str.lower()
        Valor = DataBitrix['id'].count()
        BitrixIDmaximo = DataBitrix['id'].max()
        BitrixIDmaximo = int(DataBitrix['id'].max())


        cursor = aws.cursor()

        query = "INSERT INTO telefone (id, portal_user_id , portal_number,phone_number,call_id,external_call_id,call_category,call_duration,call_start_date,call_record_url,call_vote,cost,cost_currency,call_failed_code,call_failed_reason,crm_entity_type,crm_entity_id,crm_activity_id,rest_app_id,rest_app_name,transcript_id,transcript_pending,session_id,redial_attempt,comment,record_file_id,call_type) VALUES(%s, %s , %s, %s , %s, %s,%s, %s , %s, %s , %s, %s,%s, %s , %s, %s , %s, %s,%s, %s , %s, %s , %s, %s,%s,%s, %s)"
        for r in range(Valor):
            id = DataBitrix.iloc[r, 0]
            portal_user_id = DataBitrix.iloc[r, 1]
            portal_number = DataBitrix.iloc[r, 2]
            phone_number = DataBitrix.iloc[r, 3]
            call_id = DataBitrix.iloc[r, 4]
            external_call_id = DataBitrix.iloc[r, 5]
            call_category = DataBitrix.iloc[r, 6]
            call_duration = DataBitrix.iloc[r, 7]
            call_start_date = DataBitrix.iloc[r, 8]
            call_record_url = DataBitrix.iloc[r, 9]
            call_vote = DataBitrix.iloc[r, 10]
            cost = DataBitrix.iloc[r, 11]
            cost_currency = DataBitrix.iloc[r, 12]
            call_failed_code = DataBitrix.iloc[r, 13]
            call_failed_reason = DataBitrix.iloc[r, 14]
            crm_entity_type = DataBitrix.iloc[r, 15]
            crm_entity_id = DataBitrix.iloc[r, 16]
            crm_activity_id = DataBitrix.iloc[r, 17]
            rest_app_id = DataBitrix.iloc[r, 18]
            rest_app_name = DataBitrix.iloc[r, 19]
            transcript_id = DataBitrix.iloc[r, 20]
            transcript_pending = DataBitrix.iloc[r, 21]
            session_id = DataBitrix.iloc[r, 22]
            redial_attempt = DataBitrix.iloc[r, 23]
            comment = DataBitrix.iloc[r, 24]
            record_file_id = DataBitrix.iloc[r, 25]
            call_type = DataBitrix.iloc[r, 26]

            values = (id, portal_user_id, portal_number, phone_number, call_id, external_call_id, call_category, call_duration, call_start_date, call_record_url, call_vote, cost, cost_currency, call_failed_code, call_failed_reason, crm_entity_type, crm_entity_id, crm_activity_id, rest_app_id, rest_app_name, transcript_id, transcript_pending, session_id, redial_attempt, comment, record_file_id, call_type)
            cursor.execute(query, values)
            aws.commit()


            aws = psycopg2.connect(
            host="teste.cpdbmdlpalvb.us-east-1.rds.amazonaws.com",
            database="etl_fazcapital",
            user="fazcapital",
            password="A1b2c3d4!#$%&*",
            port="5432")





        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        cursor.close()
        aws.close()

                
            
            
            
            
            
            
            
            
            
            

     
     
     
     
     
     
     
     
     
     

         
        
        

        
            
            
            
   

KeyError: ignored